In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
#import necessary libraries
import pandas as pd
import h5py
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore,kstest
from scipy.ndimage import gaussian_filter
from scipy.signal import find_peaks
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
import random
import os
import h5py

#import my functions
from placecode.spatial_coding_functions import firing_rate_map, read_spatial
from placecode.utils import open_file, open_dir
from placecode.expinfo import ExpInfo

## Locate files

In [ ]:
fpath_expinfo = open_file("Select experiment info json file!")

In [ ]:
exp_info = ExpInfo(fpath_expinfo)

In [ ]:
# make sure the necessary files exist
assert os.path.exists(exp_info.fpath_caim)
assert os.path.exists(exp_info.fpath_loco)

In [ ]:
# select folder to save results
output_folder = open_dir("Select folder for output")

## Create dict to record analysis parameters

In [ ]:
dict_pc_params = dict()

## Load data

In [ ]:
# load CaImAn data
with h5py.File(exp_info.fpath_caim, "r") as hf_caim:
    # temporal
    temporal_raw=hf_caim['estimates']['C'][()]
    n_components, n_frames = temporal_raw.shape
    resolution = hf_caim["dims"][()]
    # access a single temporal component as temporal_raw[i]

    # spatial
    resolution = hf_caim["dims"][()]
    A_data = hf_caim["estimates"]["A"]["data"][()]
    A_indices = hf_caim["estimates"]["A"]["indices"][()]
    A_indptr = hf_caim["estimates"]["A"]["indptr"][()]
    A_shape = hf_caim["estimates"]["A"]["shape"][()]
    spatial = read_spatial(A_data, A_indices, A_indptr, A_shape, n_components, resolution, unflatten=True)

In [ ]:
# load loco data
# TODO: include stripes, distance per round, etc. in loco data cut to scanner time frame. Missing in Martin's code?
#   check https://github.com/mitlabence/matlab-2p/issues/11
dict_loco = dict()
with h5py.File(exp_info.fpath_loco, "r") as hf_loco:
    for dset_name in hf_loco["inferred"]["belt_scn_dict"].keys():
        dtype = np.int16 if dset_name in ["round", "rounds", "stripes"] else np.float64
        dict_loco[dset_name] = hf_loco["inferred"]["belt_scn_dict"][dset_name][()].astype(dtype)
print(dict_loco.keys())


## Preprocess data

Create z-score of temporal components

In [ ]:
temporal_z = zscore(temporal_raw, axis=1) 

### Filter rounds
Only use rounds where the total length adds up to the expected belt length

In [ ]:
expected_distance=exp_info.belt_length_mm
lv_rounds = dict_loco["rounds"]  
lv_distPR = dict_loco["distance"] # belt_scn_dict hast distance per round as distance, see issue above
n_rounds=lv_rounds.max()  # number of finished rounds
rounds = []
round_flags = np.zeros(n_rounds, dtype=np.int8)  # 1 if corresponding round included in analysis, 0 otherwise

for round in range(1,n_rounds+1):
    dist_current_round=lv_distPR[lv_rounds==round][-1]
    #print(dist_current_round)
    if abs(dist_current_round-expected_distance)<15:
        rounds.append(round)
        round_flags[round-1] = 1

num_rounds=len(rounds)
print(f'Rounds: {rounds}\n Number of rounds used: {num_rounds}, total {n_rounds}')

# save as parameter a binary array on which round was included
dict_pc_params["rounds_included"] = round_flags
dict_pc_params["n_rounds_included"] = num_rounds

# filter the data
included_rounds_frames =  np.isin(dict_loco["rounds"], np.array(rounds) - 1)  # convert rounds to 0-indexing, filter to only those rounds that count
for k in dict_loco:
    dict_loco[k] = dict_loco[k][included_rounds_frames]
temporal_raw = temporal_raw[:, included_rounds_frames]
temporal_z = temporal_z[:, included_rounds_frames]


### Filter forward-locomoting frames
Only use frames where mouse is running forward

In [ ]:
loco_frames = dict_loco["speed"] > 0
for k in dict_loco:
    dict_loco[k] = dict_loco[k][loco_frames]
temporal_raw = temporal_raw[:, loco_frames]
temporal_z = temporal_z[:, loco_frames]

### Calculate spatial firing map

In [ ]:
n_bins = exp_info.n_bins
n_units = n_components

In [ ]:
firing_rate_map_raw = firing_rate_map(temporal_raw, dict_loco["rounds"], dict_loco["distance"], n_bins)
firing_rate_map_z = firing_rate_map(temporal_z, dict_loco["rounds"], dict_loco["distance"], n_bins)
